In [1]:
!pip install tensorflow opencv-python matplotlib

In [2]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.1
aiosignal                        1.3.1
alabaster                        0.7.13
albumentations                   1.3.1
altair                           4.2.2
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.1.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.13.1
backcall                         0.2.0
beautifulsoup4                   4.11.2
bidict                           0.22.1
b

In [3]:
import tensorflow as tf
import os

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu,True)
#This code intend to avoid OOM(Out Of Memory)Errors

**Load Data from Drive**

In [5]:
!gdown --id 1GUWUhJZPBGE7bjn6V_17ptV8a5-9kmou
!unzip green_bean_dataset.zip

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1GUWUhJZPBGE7bjn6V_17ptV8a5-9kmou
To: /content/green_bean_dataset.zip
100% 42.3M/42.3M [00:00<00:00, 245MB/s]
Archive:  green_bean_dataset.zip
   creating: green_bean_dataset/
   creating: green_bean_dataset/defect/
  inflating: green_bean_dataset/defect/BrocadoLeve - 1.jpg  
  inflating: green_bean_dataset/defect/BrocadoLeve - 10.jpg  
  inflating: green_bean_dataset/defect/BrocadoLeve - 11.jpg  
  inflating: green_bean_dataset/defect/BrocadoLeve - 12.jpg  
  inflating: green_bean_dataset/defect/BrocadoLeve - 13.jpg  
  inflating: green_bean_dataset/defect/BrocadoLeve - 14.jpg  
  inflating: green_bean_dataset/defect/BrocadoLeve - 15.jpg  
  inflating: green_bean_dataset/defect/BrocadoLeve - 16.jpg  
  inflating: green_be

**Splitting The Dataset Into Training and Validation**

In [6]:
import shutil
from sklearn.model_selection import train_test_split

# Path ke folder dataset yang sudah diekstrak
dataset_path = '/content/green_bean_dataset'

# Path ke folder training dan validation
train_dir = '/content/data_use/training'
validation_dir = '/content/data_use/validation'

# Mendefinisikan path untuk class 'defect' dan 'good'
defect_path = os.path.join(dataset_path, 'defect')
good_path = os.path.join(dataset_path, 'good')

# Membuat folder training dan validation untuk setiap class
train_defect_dir = os.path.join(train_dir, 'defect')
train_good_dir = os.path.join(train_dir, 'good')
validation_defect_dir = os.path.join(validation_dir, 'defect')
validation_good_dir = os.path.join(validation_dir, 'good')

os.makedirs(train_defect_dir, exist_ok=True)
os.makedirs(train_good_dir, exist_ok=True)
os.makedirs(validation_defect_dir, exist_ok=True)
os.makedirs(validation_good_dir, exist_ok=True)

# Mendapatkan list file untuk class 'defect' dan 'good'
defect_files = os.listdir(defect_path)
good_files = os.listdir(good_path)

# Melakukan split dataset menjadi training dan validation (misalnya, 80-20)
train_defect_files, val_defect_files = train_test_split(defect_files, test_size=0.2, random_state=42)
train_good_files, val_good_files = train_test_split(good_files, test_size=0.2, random_state=42)

# Memindahkan file ke dalam folder training dan validation
for file in train_defect_files:
    shutil.copy(os.path.join(defect_path, file), os.path.join(train_defect_dir, file))
for file in val_defect_files:
    shutil.copy(os.path.join(defect_path, file), os.path.join(validation_defect_dir, file))

for file in train_good_files:
    shutil.copy(os.path.join(good_path, file), os.path.join(train_good_dir, file))
for file in val_good_files:
    shutil.copy(os.path.join(good_path, file), os.path.join(validation_good_dir, file))

# Check jumlah gambar di setiap kelas untuk training dan validation
print(f"Jumlah gambar defect untuk training: {len(os.listdir(train_defect_dir))}")
print(f"Jumlah gambar good untuk training: {len(os.listdir(train_good_dir))}")
print(f"Jumlah gambar defect untuk validation: {len(os.listdir(validation_defect_dir))}")
print(f"Jumlah gambar good untuk validation: {len(os.listdir(validation_good_dir))}")


Jumlah gambar defect untuk training: 322
Jumlah gambar good untuk training: 323
Jumlah gambar defect untuk validation: 81
Jumlah gambar good untuk validation: 81


Augmentation For The Image

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Applying Data Augmentation for The Data
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1/255)

In [9]:
train_dir = '/content/data_use/training'
validation_dir = '/content/data_use/validation'


# Applying Data Augmentation for Training Data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')  # Menggunakan class_mode='binary' untuk klasifikasi biner

# Applying Data Augmentation for Validation Data
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary')  # Juga menggunakan class_mode='binary' untuk klasifikasi biner


Found 645 images belonging to 2 classes.
Found 162 images belonging to 2 classes.
